In [6]:
import numpy as np
import time
import os
%matplotlib qt
import matplotlib.pyplot as pl
import shutil

In [3]:
from sep_decoder import transfer,ht,lp,load_measurement_data,load_transfer_matrices,initial_state

In [25]:
def sep_dynamics(data,Q):
    (depth,L) = data.shape  
    Q2 = Q+1
    if Q>L//2:
        Q2 = Q-1
    hash_table,_ = ht.get_hash_table(L,Q)
    hash_table2,_= ht.get_hash_table(L,Q2)
    pairings = lp.local_pairings(L,Q,hash_table)
    pairings2 = lp.local_pairings(L,Q2,hash_table2)
    dim = len(hash_table)
    dim2 = len(hash_table2)
    
    total = 0

    transfer_matrices = load_transfer_matrices(pairings,dim,L,Q)
    transfer_matrices2 = load_transfer_matrices(pairings2,dim2,L,Q2)
    
    initial_state_Q = np.eye(dim)/dim**0.5
    initial_state_Q2 = np.eye(dim2)/dim2**0.5

    N=1
    traj = data
    state_Q_is_zero = False
    state_Q2_is_zero = False
    state_Q = initial_state_Q.copy()
    state_Q2 = initial_state_Q2.copy()

    log_Z = []
    log_Z2 = []
    total += N
    for t in range(depth)[:]:
        
        if t%2 == 0: #even layer
            for x in range(0,L-1,2):
                outcomes = (traj[t,x],traj[t,x+1])
            
                state_Q, state_Q_is_zero = transfer(x,dim,pairings,transfer_matrices, outcomes,state_Q,log_Z,state_Q_is_zero,L,Q,debug=False)

                state_Q2, state_Q2_is_zero = transfer(x,dim2,pairings2,transfer_matrices2 ,outcomes,state_Q2,log_Z2,state_Q2_is_zero,L,Q2)
                
                if state_Q_is_zero and state_Q2_is_zero:
                    print('hurray2',np.sum(state_Q2),np.sum(state_Q))
        else:
            for x in range(1,L-1,2):
                outcomes = (traj[t,x],traj[t,x+1])
                debug=False

                state_Q, state_Q_is_zero = transfer(x,dim,pairings,transfer_matrices, outcomes,state_Q,log_Z,state_Q_is_zero,L,Q,debug=False)

                state_Q2, state_Q2_is_zero = transfer(x,dim2,pairings2,transfer_matrices2 ,outcomes,state_Q2,log_Z2,state_Q2_is_zero,L,Q2)

                if state_Q_is_zero and state_Q2_is_zero:
                    print('hurray2',np.sum(state_Q2),np.sum(state_Q))
                

        # print(t,time.time()-start,total)
        if state_Q_is_zero:              
            break
    
        if state_Q2_is_zero:
            break
        
    if state_Q_is_zero:
        p_Q = 0
    elif state_Q2_is_zero:
        p_Q = 1
    else:
        ratio_p = np.exp(np.sum(log_Z) - np.sum(log_Z2))
        p_Q = 1/(1+1/ratio_p)
        p_Q2 = 1/(1+ratio_p)
    
    return p_Q

In [3]:
def get_p_L(filename):
    temp = filename.split(',')
    p = float(temp[-1][:-4])
    L = int(temp[1])
    return p,L

filedir = 'U_1_data_Fergus/'
p_set = set()
for file in os.listdir(filedir)[:]:
    temp = file.split(',')
    p,L = get_p_L(file)
    p_set.add(p)
p_list = list(sorted(p_set))[2::]
print(p_list,len(p_list),sorted(p_set))

[0.05, 0.0625, 0.075, 0.0875, 0.1, 0.1125, 0.125, 0.1375, 0.15, 0.1625, 0.175, 0.1875, 0.2, 0.2125, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.4, 0.45, 0.5] 23 [0.025, 0.0375, 0.05, 0.0625, 0.075, 0.0875, 0.1, 0.1125, 0.125, 0.1375, 0.15, 0.1625, 0.175, 0.1875, 0.2, 0.2125, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.4, 0.45, 0.5]


In [8]:
L_list = [6,8,10,12]
for L in L_list:
    for p in p_list:
        for file in os.listdir(filedir)[:]:
            if file[0]=='u':
                continue
            p_data,L_data = get_p_L(file)
            if p_data != p or L_data != L:
                continue
            shutil.copy(filedir+file,filedir+'useful/'+file)

: 

In [ ]:
def get_sep_data(N_samples,L,p):
    p_suc = {}
    for file in os.listdir(filedir)[:]:
        p_data,L_data = get_p_L(file)
        if p_data != p or L_data != L:
            continue
        data_raw = np.load(filedir+file)[:N_samples]
        for i in [0,1]:
            Q = int(np.sum(0.5*(data_raw[0,i,:,-1]+1)))
            p_suc[Q] = []
            for j in range(N_samples):
                p_suc[Q].append(sep_dynamics(data_raw[j,i,:,:-1].T,Q))
    return p_suc



In [54]:
L_list = [6,8,10,12]
p_suc_dic = {}
for L in L_list:
    p_suc_dic[L]={}
    for p in p_list[:-1]:
        start = time.time()
        p_suc_dic[L][p]=get_sep_data(100,L,p)
        print(L,p,time.time()-start)


3
20
15
4
6 0.0625 0.1512000560760498
3
4
6 0.0875 0.14135503768920898
3
4
6 0.1125 0.14199018478393555
3
4
6 0.1375 0.14308691024780273
3
4
6 0.1625 0.14416885375976562
3
4
6 0.1875 0.1427769660949707
3
4
6 0.2125 0.13942289352416992
3
4
6 0.25 0.14501309394836426
3
4
6 0.3 0.13991093635559082
3
4
6 0.35 0.12298703193664551
4
5
8 0.0625 0.34276771545410156
4
5
8 0.0875 0.35469889640808105
4
5
8 0.1125 0.3392510414123535
4
5
8 0.1375 0.3395230770111084
4
5
8 0.1625 0.3418402671813965
4
5
8 0.1875 0.34406399726867676
4
5
8 0.2125 0.3330357074737549
4
5
8 0.25 0.31836390495300293
4
5
8 0.3 0.29848599433898926
4
5
8 0.35 0.259382963180542
5
252
210
6
10 0.0625 2.3016130924224854
5
6
10 0.0875 2.345615863800049
5
6
10 0.1125 2.3722290992736816
5
6
10 0.1375 2.435070037841797
5
6
10 0.1625 2.3206522464752197
5
6
10 0.1875 2.266040086746216
5
6
10 0.2125 2.2814419269561768
5
6
10 0.25 2.20442795753479
5
6
10 0.3 1.882983922958374
5
6
10 0.35 1.6252689361572266
6
924
792
7
12 0.0625 78.088536

KeyboardInterrupt: 

In [57]:
def plot_success_ent(data,L_list,p_list,N_samples=-1):
    ent = {}
    err = {}
    for L in L_list[:]:
        ent[L] = []
        err[L] = []
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2+1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent.append(np.sum(np.array(suc_list)>0.5)/len(suc_list))
            ent_list = [(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ]
            ent[L].append(np.average(ent_list))
            err[L].append(np.std(ent_list)/(len(ent_list)-1)**0.5)

        pl.errorbar(p_list,ent[L],yerr=err[L],ls='-',marker='o',label='L='+str(L))

    pl.xlabel(r'$p$',fontsize=16)
    pl.ylabel(r'$S_{suc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()

In [58]:
plot_success_ent(p_suc_dic,L_list[:-1],p_list[:-1])

L= 6 p= 0.0625 data_size: 99 99
L= 6 p= 0.0875 data_size: 99 99
L= 6 p= 0.1125 data_size: 99 99
L= 6 p= 0.1375 data_size: 99 99
L= 6 p= 0.1625 data_size: 99 99
L= 6 p= 0.1875 data_size: 99 99
L= 6 p= 0.2125 data_size: 99 99
L= 6 p= 0.25 data_size: 99 99
L= 6 p= 0.3 data_size: 99 99
L= 6 p= 0.35 data_size: 99 99
L= 8 p= 0.0625 data_size: 99 99
L= 8 p= 0.0875 data_size: 99 99
L= 8 p= 0.1125 data_size: 99 99
L= 8 p= 0.1375 data_size: 99 99
L= 8 p= 0.1625 data_size: 99 99
L= 8 p= 0.1875 data_size: 99 99
L= 8 p= 0.2125 data_size: 99 99
L= 8 p= 0.25 data_size: 99 99
L= 8 p= 0.3 data_size: 99 99
L= 8 p= 0.35 data_size: 99 99
L= 10 p= 0.0625 data_size: 99 99
L= 10 p= 0.0875 data_size: 99 99
L= 10 p= 0.1125 data_size: 99 99
L= 10 p= 0.1375 data_size: 99 99
L= 10 p= 0.1625 data_size: 99 99
L= 10 p= 0.1875 data_size: 99 99
L= 10 p= 0.2125 data_size: 99 99
L= 10 p= 0.25 data_size: 99 99
L= 10 p= 0.3 data_size: 99 99
L= 10 p= 0.35 data_size: 99 99


In [2]:
850*25/3600

5.902777777777778